In [1]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding


In [2]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

c:\Users\wwwya\OneDrive\Desktop\finetune_llama3\.venv\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [3]:
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
DB_FAISS_PATH = 'collector/store/db_faiss'
def generate_and_save_embeddings(texts):
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                        model_kwargs={'device': 'cpu'})

    # Generate the embeddings
    doc_embeddings = [embeddings.embed_query(text) for text in texts]

    # Save the embeddings to a file
    np.save('embeddings.npy', doc_embeddings)

def load_data():
    loader = CSVLoader(file_path="data_car.csv", encoding="utf-8", csv_args={'delimiter': ','})
    data = loader.load()

    # Load the embeddings from a file
    embeddings = np.load('embeddings.npy')

    db = FAISS.from_documents(data, embeddings)
    db.save_local(DB_FAISS_PATH)

    return db


In [5]:
with open ('data.txt',encoding='utf-8') as f:
    data = f.readlines()

In [29]:
generate_and_save_embeddings(data)

c:\Users\wwwya\OneDrive\Desktop\New folder\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
def generate_and_save_embeddings(texts):
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                        model_kwargs={'device': 'cpu'})

    # Generate the embeddings
    doc_embeddings = [embeddings.embed_query(text) for text in texts]

    # Save the embeddings to a file
    np.save('embeddings.npy', doc_embeddings)


In [31]:
from langchain.docstore.document import Document


def load_data():
    loader = CSVLoader(file_path="data_car.csv", encoding="utf-8", csv_args={'delimiter': ','})
    data = loader.load()

    # Load the embeddings from a file
    embeddings = np.load('embeddings.npy')

    # Create a list of Document objects from the data
    documents = [Document(page_content=doc.page_content) for doc in data]

    db = FAISS.from_documents(documents, embeddings)
    db.save_local(DB_FAISS_PATH)

    return db


In [32]:
load_data()

AttributeError: 'numpy.ndarray' object has no attribute 'embed_documents'

In [33]:
pip install pinecone

  Using cached pinecone-4.0.0-py3-none-any.whl.metadata (16 kB)
Using cached pinecone-4.0.0-py3-none-any.whl (214 kB)
Note: you may need to restart the kernel to use updated packages.


In [38]:
from sentence_transformers import SentenceTransformer
import os
from pinecone import Pinecone ,ServerlessSpec
retrever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base",device='cpu')
pc = Pinecone(api_key="33fd2367-2090-4ff2-b22f-cc2503c2e1ee")
index_name = "eurotechuser"
index = pc.Index(index_name)
retrever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base",device='cpu')
pc = Pinecone(api_key="33fd2367-2090-4ff2-b22f-cc2503c2e1ee")
def query_pinecone(query,top_k):
    xq = retrever.encode([query]).tolist()
    xc = index.query(vector=xq, top_k=top_k, include_metadata=True)
    return xc
def format_query(query,context):
    context = [f"<P> {m['metadata']['paragraph']}" for m in context]
    context = " ".join(context)
    query =  f"Querstion: {query} context: {context}"
    return query

In [39]:
query = "i want to know about the policys of eurotech"
result = query_pinecone(query,top_k=3)
result

{'matches': [{'id': '502',
              'metadata': {'heading': '',
                           'paragraph': 'If you have any questions about this '
                                        "policy or Eurotech Xchange's privacy "
                                        'practices, please contact Eurotech '
                                        'Xchange.'},
              'score': 0.614123,
              'values': []},
             {'id': '484',
              'metadata': {'heading': '',
                           'paragraph': "on Eurotech Xchange's website to "
                                        'other personal information you may '
                                        'provide to Eurotech Xchange at the'},
              'score': 0.587626278,
              'values': []},
             {'id': '472',
              'metadata': {'heading': '',
                           'paragraph': 'information about you as described in '
                                        'this policy, which E